<a href="https://colab.research.google.com/github/evmpython/Minicurso_queimadas_UNIFEI_INPE_NOTTUS_2026/blob/main/AULA_1_Queimadas_com_Dados_do_INPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

#  🟢 **Minicurso** - Processamento e Visualização de Dados de Queimadas (PyFIRE)

---

> ## **Aula 1:** Processamento e Visualização de Dados de Focos de Calor Disponibilizados pelo INPE

---

**OBJETIVO:**
- Aprender a **acessar**, **analisar** e **plotar figuras** dos dados de focos de calor estimado por satélites.

---

**DADOS DE ENTRADA**:
- Serão utilizados [dados](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/) de focos de calor processados e disponibilizados pelo INPE. Os dados estão disponibilizados em quatro agrupamentos temporais:
    1. **10min**: dados dos últimos 4 dias. Exemplo: *focos_10min_20250810_0000.csv*
    2. **anual**: dados desde 2003. Exemplo: *focos_br_ref_2003.zip*
    3. **diário**: dados do mês atual. Exemplo: *focos_diario_br_20250713.csv*
    4. **mensal**: dados do ano anterior e atual. Exemplo: *focos_mensal_br_202301.zip*

---

**DADOS DE SAÍDA:**
- **Tipo do dado:** Figuras
- **Formato do dado:** JPG
- **Imagens geradas:**
    1.  Parte_2_Fig_1_mapainterativo_focos_estado.html
    2.  Parte_4_Fig_acumulado_focos_calor_Brasil_2024_e_2025.jpg
    3.  Parte_5_acumulado_e_anomalia_focos_2025.png
    4.  Parte_6_Fig_1_focos_por_dia.jpg
    5.  Parte_6_Fig_2_heatmap.jpg
    6.  Parte_6_Fig_3_focos_por_ano.jpg
    8.  Parte_6_Fig_4_calendarplot.jpg
    9.  Parte_7_risco_fogo_observado_20260204.jpg
    10. Parte_8_risco_fogo_previsto_a_partir_de_20260205.jpg

---

**PROCEDIMENTO REALIZADO:**
- Os seguintes procedimentos são realizados nesse código:
1.  Instalação das bibliotecas
2.  Importação das bibliotecas
3.  Criando Diretório de Entrada e Saída
4.  Definindo Funções
5.  Conhecendo e Acessando os Dados de Focos de Calor Fornecidos pelo INPE
6.  Leitura dos Dados de Focos de Calor de 2003 à 2026
7.  Mapa de Acumulado de Focos de Calor
8.  Mapa de Acumulado e Anomalia de Focos de Calor
9.  Séries Temporais de Focos de Calor
10. Risco de Fogo Observado
11. Risco de Fogo Previsto

---
**OBSERVAÇÕES IMPORTANTES**:
1. Necessário possuir uma conta do Gmail.

---
**EQUIPE:**
- Palestrantes/Tutores:

  - Enrique Vieira Mattos - UNIFEI: enrique@unifei.edu.br / https://github.com/evmpython
  - Guilherme Martins - Nottus Meteorologia: jgmasantos@gmail.com / https://github.com/jgmsantos
  - Diego Souza - INPE: diego.souza@inpe.br / https://github.com/diegormsouza

- Colaboradores:
  - Diego Souza - INPE: diego.souza@inpe.br / https://github.com/diegormsouza

 ---

![Texto alternativo](https://github.com/evmpython/Minicurso_queimadas_UNIFEI_INPE_2026/blob/main/04_logos/resumo_figuras_aula1.png?raw=true)

Observações:
1. Horário dentro do arquivo é UTC?

# **1° Passo:** Instalando as Bibliotecas

O Google Colab roda uma máquina Linux na Nuvem que já tem o Python instalado. Vamos verificar a versão do python que temos disponível.

In [ ]:
# verificando a versão do python instalado no Google Colab
!python --version

O primeiro passo será instalar as bibliotecas necessárias (e suas dependências) para a execução dos códigos. Basicamente, as bibliotecas terão a seguinte finalidade:

*   `ultraplot:` Plota gráficos de alta qualidade
*   `cartopy:` Trabalha com mapas
*   `calplot:` Plota figura em forma de calendário (anos x meses)
*   `leafmap:` Mapa interativo
*   `salem:` Extrair dados de um shapefile
*   `rasterio, pyproj, geopandas e descartes:` Extrair informações de uma região

In [ ]:
# verificando as bibliotecas instaladas no Colab
!pip list

In [ ]:
# verificando se a biblioteca "leafmap" esta instalada no Colab
!pip show leafmap

In [ ]:
!pip install -q ultraplot cartopy calplot leafmap rasterio pyproj geopandas salem descartes xarray rioxarray

In [ ]:
# verificando se a biblioteca "leafmap" foi instalada no Colab
!pip show leafmap

# **2° Passo:** Importando as Bibliotecas

In [ ]:
#========================================================================================================================#
#                 BIBLIOTECAS PARA MANIPULAÇÃO DE ARQUIVOS, SISTEMA E OPERAÇÕES BÁSICAS
#========================================================================================================================#
import os                                   # Interação com sistema operacional: caminhos, diretórios, variáveis de ambiente
import glob                                 # Lista arquivos em diretórios usando padrões (ex: *.nc)
import time                                 # Medição de tempo de execução e operações relacionadas a tempo
import json                                 # Codificação e decodificação de dados no formato JSON
import warnings                             # Gerenciamento de mensagens de alerta durante a execução
warnings.filterwarnings("ignore")           # Supressão de avisos para manter a saída limpa

#========================================================================================================================#
#                          BIBLIOTECAS PARA MANIPULAÇÃO DE DADOS TEMPORAIS
#========================================================================================================================#
from datetime import datetime               # Manipulação de datas e horas
import calendar                             # Operações relacionadas a calendários
import pytz                                 # Conversão de fusos horários (UTC para local e vice-versa)

#========================================================================================================================#
#                        BIBLIOTECAS PARA COMUNICAÇÃO WEB E REQUISIÇÕES HTTP
#========================================================================================================================#
import requests                             # Realização de requisições HTTP para APIs e downloads web

#========================================================================================================================#
#                    BIBLIOTECAS PARA COMPUTAÇÃO NUMÉRICA E MANIPULAÇÃO DE DADOS
#========================================================================================================================#
import numpy as np                          # Operações matemáticas eficientes com arrays multidimensionais
import pandas as pd                         # Manipulação e análise de dados em estruturas tabulares (DataFrames)
import xarray as xr                         # Trabalho com arrays multidimensionais rotulados (comum em dados científicos)

#========================================================================================================================#
#                       BIBLIOTECAS PARA DADOS GEOESPACIAIS E GEORREFERENCIAMENTO
#========================================================================================================================#
import geopandas as gpd                     # Manipulação de dados vetoriais geoespaciais (extensão do pandas)
import rioxarray                            # Integração de funcionalidades rasterio com xarray para dados geoespaciais
import salem                                # Integração de dados geoespaciais com xarray (especializado em climatologia)

#========================================================================================================================#
#                         BIBLIOTECAS PARA VISUALIZAÇÃO CARTOGRÁFICA E MAPAS
#========================================================================================================================#
import cartopy                              # Ferramentas para criação de mapas e projeções cartográficas
import cartopy.crs as ccrs                  # Sistemas de referência de coordenadas para projeções de mapas
import cartopy.io.shapereader as shpreader  # Leitura de arquivos shapefile para mapas
import leafmap                              # Criação interativa de mapas e visualização de dados geoespaciais
#import leafmap.foliumap as leafmap

#========================================================================================================================#
#                       BIBLIOTECAS PARA VISUALIZAÇÃO DE DADOS E GRÁFICOS
#========================================================================================================================#
import matplotlib.pyplot as plt             # Criação de gráficos e visualizações 2D/3D (biblioteca base)
import seaborn as sns                       # Visualização estatística de dados (baseado no matplotlib)
import calplot                              # Criação de calendários de visualização de dados temporais
import ultraplot as uplt                    # Visualização especializada para dados meteorológicos/climáticos

# **3° Passo:** Criando Diretório de Entrada e Saída

Neste etapa criaremos os diretórios `input`, para onde iremos baixar os dados de entrada e `output`, onde estarão as figuras que serão produzidas.

![Texto alternativo](https://github.com/evmpython/Minicurso_queimadas_UNIFEI_INPE_2026/blob/main/04_logos/input_output.png?raw=true)

In [ ]:
# cria a pasta de entrada com o nome "input"
os.makedirs(f'/content/input', exist_ok=True)

# cria a pasta de saída com o nome "output"
os.makedirs(f'/content/output', exist_ok=True)

# **4° Passo:** Definindo Funções

Função que contabiliza a quantidade de focos de calor por pixel

In [ ]:
# Função que calcula o índice i e j da localização do foco de calor
def index(longitudes_matriz, latitudes_matriz, lon_foco, lat_foco):

    ''' Função para calcular o índice (i e j) do pixel de uma matriz que o relâmpago pertence

    Parâmetros:
               longitudes_matriz (array): array de uma dimensão das longitudes da matriz em graus
               latitudes_matriz (array): array de uma dimensão das latitudes da matriz em graus
               lon_raio (float): valor da longitude do foco em graus
               lat_raio (float): valor da latitude do foco em graus

    Retorna:
            indice_lat_raio (float): índice da latitude (ou seja, da linha) do pixel da matriz que o foco pertence
            indice_lon_raio (float): índice da longitude (ou seja, da coluna) do pixel da matriz que o foco pertence
    '''

    # calcula a diferença entre as lats/lons da matriz e a latitude/longitude do foco
    distancia_lon = (longitudes_matriz - lon_foco)**2
    distancia_lat = (latitudes_matriz - lat_foco)**2

    # índice da longitude e latitude do foco
    indice_lon_foco = np.nonzero(distancia_lon == np.min(distancia_lon))
    indice_lat_foco  = np.nonzero(distancia_lat == np.min(distancia_lat))

    # retorna os valores dos índices calculados
    return indice_lat_foco, indice_lon_foco

# **PARTE 1):** Conhecendo e Acessando os Dados de Focos de Calor Fornecidos pelo INPE

![Texto alternativo](https://github.com/evmpython/Minicurso_queimadas_UNIFEI_INPE_2026/blob/main/04_logos/dadosinpe.png?raw=true)

O INPE processa e fornece diversos tipos de dados e produtos relacionados a queimadas estimados por satélites polares e geoestacionários. Estas informações são disponibilizadas através do Portal [terrabrasilis](https://terrabrasilis.dpi.inpe.br/queimadas/portal/) e existem diversas maneiras de utilizar os diferentes tipos de dados. Por exemplo, a visualização em mapa interativo e download dos dados pode ser realizada através do [BDQueimadas](https://terrabrasilis.dpi.inpe.br/queimadas/bdqueimadas/). Por outro lado, o INPE também fornece os [dados](https://terrabrasilis.dpi.inpe.br/queimadas/portal/pages/secao_downloads/dados-abertos/#da-focos) em diferentes formatos (CSV, TIF) de:

1. [Focos de queimadas](https://terrabrasilis.dpi.inpe.br/queimadas/portal/pages/secao_downloads/dados-abertos/#da-focos)
2. [Área queimada](https://terrabrasilis.dpi.inpe.br/queimadas/portal/pages/secao_downloads/dados-abertos/#da-area-qmd)
3. [Risco de fogo](https://terrabrasilis.dpi.inpe.br/queimadas/portal/pages/secao_downloads/dados-abertos/#da-rf)

Para o nosso curso utilizaremos os dados de ocorrência de focos de calor em formato CSV disponíveis no [FTP](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/) do INPE. Neste FTP os dados de focos de calor estão agrupados em quatro escalas temporais:

1. [`10min`](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/10min/): dados com frequência temporal de 10 min contendo os focos de calor detectados por todos os satélites para a América do Sul. Exemplo: *focos_10min_20260131_0000.csv*

2. [`anual`](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/): dados agrupados por ano e estão disponíveis desde 2003. São disponibilizados focos de calor para: i) América do Sul detectados pelo satélite de referência (*AMS_sat_ref*), ii) Brasil detectados pelo satélite de referência (*Brasil_sat_ref*), iii) Brasil detectados por todos os satélites (*Brasil_todos_sats*) e iv) estados brasileiros detectados pelo satélite de referência (*EstadosBr_sat_ref*).  

3. [`diario`](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/): dados agrupados por dia e estão disponíveis para o mês atual. São disponibilizados focos de calor para: i) América do Sul detectados por todos os satélites (*America_Sul*) e ii)Brasil detectados por todos os satélites (*Brasil*).

4. [`mensal`](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/): dados agrupados por mês e estão disponíveis desde 2023. São disponibilizados focos de calor para: i) América do Sul detectados por todos os satélites (*America_Sul*) e ii)Brasil detectados por todos os satélites (*Brasil*).

Acessando e visualizando os dados a cada [`10min`](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/10min/): iremos analisar os dados de focos de calor que ocorreram no dia *3 de fevereiro de 2026 às 13:10 UTC* detectados por todos os satélites abrangendo a América do Sul.

In [ ]:
# link dos dados do INPE
url_10min = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/10min/focos_10min_20260205_1010.csv'

In [ ]:
# leitura dos dados
focos = pd.read_csv(url_10min)

In [ ]:
# mostrando os dados
focos

A planilha dos dados possui 4 colunas contendo as seguintes informações:

1. `lat`: latitude em graus da localização do foco de calor
2. `lon`: longitude em graus da localização do foco de calor
3. `satelite`: nome do satélite que detectou o foco de calor
4. `data`: data no formato *ano-mês-dia hora:minuto:segundo* do foco de calor

Veja que neste arquivo de dados a cada 10min temos a detecção de focos de calor realizada por todos os satélites. Para verificar os nomes dos satélites disponíveis na coluna `satelite`, utilizaremos o comando [unique()](https://pandas.pydata.org/docs/reference/api/pandas.unique.html) do pandas. A função unique encontra os valores/nomes únicos (ou seja, que não se repetem) numa determinada lista de valores.

In [ ]:
focos['satelite'].unique()

Então neste arquivo os satélites que detectaram focos de calor foram o `AQUA`, `SUOMI-NPP` e `NOOA-21`. Baseado no horário do arquivo e no horário de passagem do satélite, os satélites que detectam foco de calor pode variar.

Acessando e visualizando os dados [`anual`](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/): iremos analisar os dados de focos de calor de *2023* para o Brasil detectados pelo satélite de referência.

In [ ]:
# link dos dados do INPE
url_anual = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/Brasil_sat_ref/focos_br_ref_2023.zip'

In [ ]:
# leitura dos dados
focos = pd.read_csv(url_anual, compression='zip')

In [ ]:
# mostrando os dados
focos

A planilha dos dados possui 9 colunas contendo as seguintes informações:

1. `id_bdq`: ID do foco de calor no BDqueimadas
2. `foco_id`: identificador do foco de calor
3. `lat`: latitude em graus da localização do foco de calor
4. `lon`: longitude em graus da localização do foco de calor
5. `data_pas`: data no formato ano-mês-dia hora:minuto:segundo do foco de calor
6. `pais`: país onde ocorreu o foco de calor
7. `estado`: estado brasileiro onde ocorreu o foco de calor
8. `municipio`: município brasileiro onde ocorreu o foco de calor
9. `bioma`: bioma brasileiro onde ocorreu o foco de calor


Acessando e visualizando os dados a cada [`diario`](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/): iremos analisar os dados de focos de calor que ocorreram no dia *2 de fevereiro de 2026* no Brasil detectados por todos satélites.

In [ ]:
# link dos dados do INPE
url_diario = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/Brasil/focos_diario_br_20260202.csv'

In [ ]:
# leitura dos dados
focos = pd.read_csv(url_diario)

In [ ]:
# mostrando os dados
focos.head(2)

A planilha dos dados possui 16 colunas contendo as seguintes informações:

1. `id`: identificador do foco de calor
3. `lat`: latitude em graus da localização do foco de calor
4. `lon`: longitude em graus da localização do foco de calor
5. `data_hora_gmt`: data no formato ano-mês-dia hora:minuto:segundo em UTC do foco de calor
6. `satelite`: satélite que detectou o foco de calor
7. `municipio`: município brasileiro onde ocorreu o foco de calor
8. `estado`: estado brasileiro onde ocorreu o foco de calor
9. `pais`: país onde ocorreu o foco de calor
10. `municipio_id`: número identificador do município
11. `estado_id`: número identificador do estado
12. `pais_id`: número identificador do país
13. `numero_dias_sem_chuva`: número de dias sem chuva
14. `precipitacao`: intensidade da taxa de precipitação
15. `risco_fogo`: risco de fogo
16. `bioma`: bioma onde ocorreu o foco de calor
17. `frp`: Fire Radiative Power ou Potência Radiativa do Fogo mede a energia radiante liberada por unidade de tempo, associada à taxa de queima da vegetação em MW (megawatts)

Acessando e visualizando os dados a cada [`mensal`](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/): iremos analisar os dados de focos de calor detectados em janeiro de 2026 no Brasil por todos os satélites.

In [ ]:
# link dos dados do INPE
url_mensal = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/Brasil/focos_mensal_br_202601.csv'

In [ ]:
# leitura dos dados
focos = pd.read_csv(url_mensal)

In [ ]:
# mostrando os dados
focos

A planilha dos dados possui 16 colunas com a mesma formatação dos dados diários explicados anteriormente.

Agora que entendemos onde podemos baixar e qual a formatação dos dados de focos de calor do INPE, iremos gerar os primeiros gráficos do nosso curso. Então vamos começar!

# **PARTE 2):** Mapas Interativos de Focos de Calor

Nesta etapa do curso iremos plotar os focos de calor num mapa interativo. A grande vantagem desta abordagem é a possibilidade de darmos um zoom numa determinada região e sabermos qual localidade foi detectado o foco de calor. Além disso poderemos avaliar a distrbuição espacial e a densidade de focos de calor nesta região. Para isto usaremos o pacote pacote chamado leafmap.

O [`leafmap`](https://leafmap.org/) é um pacote para mapeamento interativo de dados espacial em Python. Foi criado em 2021 e é extremamente versátil e amigável. Foi desenvolvido por Qiusheng Wu como um derivado do pacote `geemap`, usado exclusivamente no contexto do **Google Earth Engine**. O `leafmap` oferece uma sintaxe simples que permite a criação de mapas interativos de forma eficiente através da integração com outros pacotes como `Folium` e `ipyleaflet`.

![Texto alternativo](https://github.com/evmpython/Minicurso_queimadas_UNIFEI_INPE_2026/blob/main/04_logos/leafmap.png?raw=true)

Para criar o mapa interativo com `leafmap` utilizaremos os seguintes comandos:

 - `Map = leafmap.Map(center=(0, -60), zoom=4)`: define o mapa interativo com `leafmap.Map()`, sendo que o argumento `center` define o centro do mapa especificado como um par de coordenadas em latitude e longitude. Além disso, o argumento `zoom` controla o nível de zoom inicial do mapa.

 - `Map.add_basemap('SATELLITE')`: adiciona um mapa de base padrão à visualização que depende da extensa lista disponível no `leafmap`.

 - `Map`: exibe o mapa interativo com as configurações e camadas definidas anteriormente de forma que é possível navegar, dar zoom e explorar os dados.

In [ ]:
# define o mapa
Map = leafmap.Map(center=(0, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('SATELLITE')

# mostra o mapa
Map

Mas, por mais que o mapa seja interativo, um mapa vazio não tem sentido científico. Então vamos carregar os dados de focos de calor do INPE dos [arquivos](https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/10min/) disponibilizados a cada 10min. Iremos visualizar os focos de calor do dia *3 de fevereiro de 2026 às 13:10 UTC*.

In [ ]:
# link dos dados do INPE
url_10min = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/10min/focos_10min_20260202_1510.csv'

In [ ]:
# leitura dos dados
focos = pd.read_csv(url_10min)

In [ ]:
# mostra os dados
focos

In [ ]:
# quantidade de focos daquele horário
focos.shape[0]

In [ ]:
# nomes dos satélites
focos['satelite'].unique()

Agora que carregamos os dados de focos de calor do INPE, estamos prontos para plotar o mapa interativo. Para isto utilizaremos a função [`add_xy_data()`](https://leafmap.org/leafmap/#leafmap.leafmap.Map.add_xy_data) do leafmap, passando como argumentos a `tabela de dados` (*focos*) e indicando as colunas da matriz que são a `longitude` (*lon*) e `latitude` (*lat*).

> `Lembre-se`:
 - O nome do argumento em `x` e `y` deve ser igual ao nome da coluna de latitude e longitude da nossa tabela de dados. No nosso caso o nome das referidas colunas são `lon`e `lat`.

In [ ]:
# define o mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_xy_data(focos, x="lon", y="lat")

# exibe o mapa
Map

Ao analisar focos de calor é muito importante sabermos em qual município ele ocorreu. Para isto iremos plotar o contorno dos municípios brasileiros. Estas informações são fornecidas pelo [Geodata BR - Brasil](https://github.com/tbrugz/geodata-br) disponíveis no GitHub. Este projeto contém arquivos Geojson com os perímetros dos municípios brasileiros dividido por estado.

In [ ]:
contornos_municipios = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-100-mun.json'

Estamos interessados nos focos de calor que ocorreram no Brasil. Entretando este banco de dados contêm todos os focos de calor para todos os países da América Latina.

Para selecionar apenas os focos de calor no Brasil usaremos o arquivo json que contêm os contornos dos municípios brasileiros (*contornos_municipios*).

Através dessa abordagem, vamos *mascarar* os dados, restringindo a exibição dos focos de calor exclusivamente à área do Brasil.

O [`GeoPandas`](https://geopandas.org/en/stable/) é pacote que oferece suporte à manipulação e análise de dados
geoespaciais, combinando funcionalidades do `Pandas`. Ele pode ser usado para simplificar tarefas como a leitura de formatos de arquivos geoespaciais e a realização de operações espaciais.

Agora que importamos o geopandas, iremos ler o arquivo json dos contornos (*contornos_municipios*) dos municípios brasileiros. Para isto utilizaremos a função [gpd.read_file](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html) do geopandas.

In [ ]:
# leitura do arquivo contendo os contornos dos munícipios brasileiros
contornos_municipios_gpd = gpd.read_file(contornos_municipios)

O `GeoPandas` tem suporte a uma série de formato de arquivos geoespaciais, como shapefile (`.shp`), GeoJSON (`.geojson`) e outros. E assim como o `Pandas` tem uma estrutura de dados chamada `DataFrame`, o `GeoPandas` tem o `GeoDataFrame`:

In [ ]:
# mostrando o tipo do dado
type(contornos_municipios_gpd)

In [ ]:
# mostrando o GeoDataframe
contornos_municipios_gpd

Agora temos duas variáveis que iremos combinar:

1. `focos`: Variável contendo a latitude e longitude dos focos de calor
2. `contornos_municipios_gpd`: GeoDataFrame contendo os limites de cada município brasileiro.

Assim iremos combinar as duas variáveis e gerar um dataframe com os focos de calor e o respectivo município que ele pertence. Como primeiro passo iremos colocar a variável `focos` num GeoDataFrame, gerando a variável `focos_gpd`. Assim cada foco de calor terá uma geometria associada a sua latitude/longitude.


In [ ]:
# novo geodataframe
focos_gpd = gpd.GeoDataFrame(focos.reset_index(),
                             geometry=gpd.points_from_xy(focos.lon, focos.lat))

In [ ]:
# atribuição do sistema de coordenadas de referência (CRS)
focos_gpd.crs = contornos_municipios_gpd.crs

Como resultante, temos o seguinte novo `GeoDataFrame`:

In [ ]:
focos_gpd.head()

Lembrando que temos os municípios no formato geodataframe.

In [ ]:
contornos_municipios_gpd

Agora iremos gerar uma máscara para manter apenas os focos de calor que estão dentro dos limites do Brasil. Essa operação será realizada com a função [overlay](https://geopandas.org/en/stable/docs/reference/api/geopandas.overlay.html) através da interseção entre as variáveis `focos_gpd` e `contornos_municipios_gpd`.

In [ ]:
focos_brasil = gpd.overlay(focos_gpd,
                           contornos_municipios_gpd,
                           how='intersection')

Para finalizar, vamos ter que fazer a conversão de tempo e atribuição de um `index` para a tabela com `Pandas`:

In [ ]:
focos_brasil['data'] = pd.to_datetime(focos_brasil['data'])
focos_brasil.set_index('data', inplace=True)

Como resultante, temos que:

In [ ]:
focos_brasil

Note que agora temos os dados de focos de calor indicando a qual município brasileiro ele pertence. Por fim, vamos criar mais o mapa interativo com apenas os focos de calor que ocorreram no Brasil.

In [ ]:
# define o mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_xy_data(focos_brasil, x="lon", y="lat")

# exibe o mapa
Map

A variável `contornos_municipios` acima está recebendo uma URL que leva a um arquivo JSON hospedado no GitHub. Usando esse link, vamos adicionar os contornos dos municípios brasileiros ao mapa interativo usando. Para isto utilizaremos a função `.add_geojson()` do leafmap.

In [ ]:
# define o mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_xy_data(focos_brasil, x="lon", y="lat")

# adiciona contorno dos municípios do Brasil
Map.add_geojson(contornos_municipios, layer_name="Munícipios do Brasil")

# exibe o mapa
Map

Em alguns casos queremos adicionar apenas os focos de calor e os contornos dos municípios de um determinado estado. Por exemplo, imagine que gostaríamos de selecionar apenas os focos de calor do estado de MG. Aplicaremos procedimento semelhante ao realizado anteriormente a nível de Brasil. Veja um [código](https://leafmap.org/notebooks/50_marker_cluster/) de exemplo.

In [ ]:
#=============================================================================================#
#                                     IMPORTA BIBLIOTECAS
#=============================================================================================#
import leafmap.foliumap as leafmap
import pandas as pd
import geopandas as gpd
import os

#=============================================================================================#
#                                     DEFINE O SHAPEFILE
#=============================================================================================#
# para escolher o código do shapefile acesse: https://github.com/tbrugz/geodata-br
contornos_municipios_estado = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-43-mun.json'
contornos_estados = 'https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/brazil-states.geojson'

#=============================================================================================#
#                                   LEITURA DOS DADOS
#=============================================================================================#
# link dos dados do INPE
url_10min = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/10min/focos_10min_20260202_1510.csv'

# leitura dos dados
focos = pd.read_csv(url_10min)

#=============================================================================================#
#                                       APLICA A MÁSCARA
#=============================================================================================#
# leitura do arquivo contendo os contornos dos munícipios daquele Estado
contornos_municipios_estado_gpd = gpd.read_file(contornos_municipios_estado)

# novo geodataframe
focos_estado_gpd = gpd.GeoDataFrame(focos.reset_index(), geometry=gpd.points_from_xy(focos.lon, focos.lat))

# atribuição do sistema de coordenadas de referência (CRS)
focos_estado_gpd.crs = contornos_municipios_estado_gpd.crs

# seleciona os focos pertencentes aquele Estado
focos_estado = gpd.overlay(focos_estado_gpd, contornos_municipios_estado_gpd, how='intersection')

# conversão de tempo e atribuição de um "index" para a tabela
focos_estado['data'] = pd.to_datetime(focos_estado['data'])
focos_estado.set_index('data', inplace=True)

# seleciona os nomes dos satélites
satelites = focos_estado['satelite'].unique().tolist()

#=============================================================================================#
#                                      MAPA INTERATIVO
#=============================================================================================#
# define o mapa
Map = leafmap.Map(center=(-10, -40), zoom=4, height="600px", layers_control=True)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota contorno dos municípios
Map.add_geojson(contornos_municipios_estado, layer_name="Munícipios de MG")

# adiciona contorno dos estados
#Map.add_geojson(contornos_estados, layer_name="Estados do Brasil")

# cores dos focos por satélite
cores = ['orange', 'blue', 'yellow', 'green', 'red', 'pink'] ; cores = cores[:len(satelites)]

# ícones dos focos por satélite
icones = ['fire', 'map', 'pencil', 'map-marker', 'diamond', 'flash']  ; icones = icones[:len(satelites)]

# plota os dados
#Map.add_xy_data(focos_estado, x="lon", y="lat")
for i, satelite in enumerate(satelites):

    # focos
    xfocos = focos_estado[focos_estado['satelite'] == satelite]

    # quantidade de focos daquele satélite
    qte_focos = xfocos.shape[0]
    print(satelite, qte_focos)

    if qte_focos != 0:
        Map.add_points_from_xy(xfocos,
                               x = "lon",
                               y = "lat",
                               popup = ['lat', 'lon', 'satelite', 'name'],
                               layer_name = satelite,
                               marker_colors=[cores[i]],
                               color_column = "satelite",
                               icon_names = [icones[i]],
                               icon_colors=[cores[i]],
                               spin = True,
                               add_legend = True)

# exibe o mapa
Map

Podemos salvar o mapa num link HTML que poderá ser compartilhado. Para isto usaremos a função `.to_html` do geemap. Para maiores detalhes acesse [este](https://leafmap.org/notebooks/19_map_to_html/) exemplo.

In [ ]:
# cria diretório de saída
dir_output = '/content/output/Parte_2' ; os.makedirs(dir_output, exist_ok=True)

# salva imagem para HTML
Map.to_html(f'{dir_output}/Parte_2_Fig_1_mapainterativo_focos_estado.html')

Você pode alterar o tipo de ícones acessando a [lista](https://fontawesome.com/v4/icons/) de ícones disponiveis.

A criação de um mapa interativo com `leafmap` oferece uma ferramenta de uso simples para a visualização dos focos de calor sobre a América do Sul nas últimas horas e dias, o que nos permite identificar visualmente a distribuição geográfica desses focos e facilitando uma análise mais contextualizada das áreas afetadas.

---

>`TAREFA:`


- Escolha uma data diferente e avalie a distribuição espacial dos focos de calor.

---



# **PARTE 3):** Leitura dos Dados de Focos de Calor de 2003 à 2026

Iremos plotar o mapa de focos de calor para o Brasil para os anos de 2024 e 2025. Para isto, primeiramente iremos ler os dados de focos de calor do INPE diretamente com o pandas.

In [ ]:
%%time
#========================================================================================================================#
#                                              DADOS DE 2003-2024
#========================================================================================================================#
# São dados por `ANO` do `Brasil` do satélite de referência `AQUA`

# cria uma tabela vazia
df_2003_a_2024 = pd.DataFrame()

# FTP dos dados anual do satélite AQUA
url = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/Brasil_sat_ref/'

# ano inicial e final: neste caso de 2003 a 2024
anoi, anof = '2003', '2025'

# loop dos meses
for ano in range(int(anoi), int(anof)):

    print('Processando ===>>', ano)

    # nome do arquivo
    filename = f'focos_br_ref_{str(ano)}.zip'

    # leitura da tabela: 2023
    df0 = pd.read_csv(f'{url}focos_br_ref_{str(ano)}.zip', compression='zip')

    # junta a tabela que foi lida com a anterior
    df_2003_a_2024 = pd.concat([df0, df_2003_a_2024], ignore_index=True)

# remove colunas
df_2003_a_2024.drop(['id_bdq','foco_id','pais'], axis=1, inplace=True)

# renomeia coluna
df_2003_a_2024.rename(columns={'data_pas': 'data'}, inplace=True)

# reposiciona as colunas
df_2003_a_2024 = df_2003_a_2024[['data','lat','lon','municipio','estado','bioma']]

#========================================================================================================================#
#                                               DADOS DE 2025 e 2026
#========================================================================================================================#
# São dados por `MÊS` do `Brasil` de `TODOS` satélites

# cria uma tabela
df_2025_e_2026 = pd.DataFrame()

# FTP dos dados mensais de 2025 e 2026 do satélite AQUA
url = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/Brasil/'

# extrai o ano e mês atual
ano_atual, mes_atual = datetime.now().year, datetime.now().month

# monthrange retorna (primeiro_dia_semana, ultimo_dia)
ultimo_dia = calendar.monthrange(ano_atual, mes_atual)[1]

# monta data final
data_final = str(ano_atual) + str(mes_atual).zfill(2) + str(ultimo_dia).zfill(2)

# loop de 2025 e 2026
for data in pd.date_range('20250101', data_final, freq='1M'):

    # extrai ano e mês
    ano = data.strftime('%Y')
    mes = data.strftime('%m')

    print('Processando ===>>', ano, mes)

    # nome do arquivo
    filename = f'focos_mensal_br_{ano}{mes}.csv'

    # leitura da tabela
    df0 = pd.read_csv(f'{url}{filename}')

    # junta a tabela que foi lida com a anterior
    df_2025_e_2026 = pd.concat([df_2025_e_2026, df0], ignore_index=True)

# seleciona apenas para o satélite de referência AQUA_M-T
df_2025_e_2026 = df_2025_e_2026[ df_2025_e_2026['satelite'] == 'AQUA_M-T' ]

# renomeia coluna
df_2025_e_2026.rename(columns={'data_hora_gmt': 'data'}, inplace=True)

# reposiciona as colunas
df_2025_e_2026 = df_2025_e_2026[['data','lat','lon','municipio','estado','bioma']]

#========================================================================================================================#
#                                   JUNTA OS ARQUIVOS 2003-2024 e 2025-2026
#========================================================================================================================#
# junta os dataframes
df = pd.concat([df_2003_a_2024, df_2025_e_2026], ignore_index=True)

# transforma a coluna "datahora" para o formato "datetime"
df['data'] = pd.to_datetime(df['data'])

# seta a coluna "datahora" como o índice da tabela
df.set_index('data', inplace=True)

# ordena a tabela pelo índice
df.sort_index(inplace=True)

In [ ]:
# mostra os dados de 2003 à 2024
df_2003_a_2024

In [ ]:
# mostra os dados de 2025 à 2026
df_2025_e_2026

In [ ]:
# mostra o dataframe completo
df

# **PARTE 4):** Mapa de Acumulado de Focos de Calor

## Acumula na grade

In [ ]:
%%time
#========================================================================================================================#
#                                   DEFINE OS 2 ANOS QUE SERÃO PROCESSADOS
#========================================================================================================================#
# define os 2 anos que serão analisados
anos = ['2024', '2025']

#========================================================================================================================#
#                                          CRIA DIRETÓRIO DE SAÍDA
#========================================================================================================================#
dir_output = '/content/output/Parte_4'
os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                        DEFINE A MATRIZ DE DADOS
#========================================================================================================================#
# limites do Brasil
lonmin, lonmax, latmin, latmax = -75.0, -33.0, -35.0, 7.0

# espaçamento da grade
delta = 20/100.   # grade com 20 km de resolução espacial

# montando a grade
lons = np.arange(lonmin, lonmax, delta)
lats = np.arange(latmax, latmin, -delta)

# quantidade de pontos para longitude e latitude
nlon = len(lons)
nlat = len(lats)

#========================================================================================================================#
#                                                  LOOP NOS ANOS
#========================================================================================================================#
# loop dos anos
for ano in anos:

    print('Processando ===>>>', ano)

    # seleciona os focos daquele ANO
    df_selec = df.loc[ano]

    # gera matriz de raios
    focos_lon, focos_lat = df_selec['lon'].values, df_selec['lat'].values

    # interpolando para ponto de grade
    focos = np.zeros((nlat, nlon))

    # loop em cada longitude e latitude da lista
    for lonfoco, latfoco in zip(focos_lon, focos_lat):

        # função que extrai a qual pixel aquele relâmpago pertence
        lin, col = index(lons, lats, lonfoco, latfoco)

        # soma os relâmpagos por pixel
        focos[lin,col]+=1
    print('.... Gerou a matriz de focos de calor')

    # gera arquivo netcdf
    data_vars = {'focos':(('lat', 'lon'), focos, {'units': 'ocorrências/400km²', 'long_name':'Focos de Calor'})}
    coords = {'lat': lats, 'lon': lons}
    ds = xr.Dataset(data_vars=data_vars, coords=coords)
    ds.to_netcdf(f'{dir_output}/Parte_4_focos_anual_Brasil_AQUA_{ano}.nc')
    print('.... Gerou e salvou arquivo netcdf','\n')

## Figura

In [ ]:
%%time
#========================================================================================================================#
#                                                DEFINIÇÕES
#========================================================================================================================#
# limites do Brasil
lonmin, lonmax, latmin, latmax = -75.0, -33.0, -35.0, 7.0

# leitura do shapefile do Brasil
shapefile_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp')

#========================================================================================================================#
#                                              PLOTA FIGURA
#========================================================================================================================#
# cria a moldura da figura
fig, ax = uplt.subplots(axwidth=6, ncols=2, nrows=1, tight=True, proj='pcarree')

# define formato da figura
ax.format(coast=False, borders=False, innerborders=False,
          labels=False, latlines=5, lonlines=10,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          small='20px', large='25px',
          linewidth=0, grid=False,
          suptitle='Acumulado de Focos de Calor')

#--------------------------------------------------------------#
#                            FIG-1
#--------------------------------------------------------------#
# leitura do arquivo netcdf
ds = xr.open_dataset(f'{dir_output}/Parte_4_focos_anual_Brasil_AQUA_{anos[0]}.nc')

# total de focos
total = ds['focos'].salem.roi(shape=shapefile_brasil) ; total = int(np.sum(total))

# plota mapa
map1 = ax[0].contourf(ds['lon'],
                      ds['lat'],
                      ds['focos'].salem.roi(shape=shapefile_brasil),
                      alpha=1.0,
                      cmap='lajolla',
                      vmin=0.1, vmax=160,
                      levels=np.array([0.1, 5, 10, 15, 20, 30, 40, 50, 60, 70, 100, 130, 160]),
                      extend='max',
                      colorbar='best',
                      colorbar_kw={'label': 'Satelite: AQUA/ Resolução Mapa: 20km \nFonte Dados: INPE',
                                   'length': 18,
                                   'frameon': False,
                                   'ticklabelsize': 10,
                                   'labelsize': 7,
                                   'width': 2,
                                   'ticks': [20, 50, 100, 150],
                                   'ticklabels': ['20', '50', '100', '150']})

# plota subtítulo
ax[0].format(title=anos[0],
             titleloc='c',
             titleweight='bold',
             titlecolor='gray')

# plota contorno dos Estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/refs/heads/main/estados_do_brasil/BR_UF_2019.shp').geometries())
ax[0].add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='gray', facecolor='none', linewidth=0.3, alpha=1.0)

# plota contorno do Brasil
shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp').geometries())
ax[0].add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.0, alpha=1.0)

# plota quantidade de focos de calor na legenda
ax[0].text(-46, -30., f'Total={total}', color='red', fontsize=15)

#--------------------------------------------------------------#
#                            FIG-2
#--------------------------------------------------------------#
# leitura do arquivo netcdf
ds = xr.open_dataset(f'{dir_output}/Parte_4_focos_anual_Brasil_AQUA_{anos[1]}.nc')

# total de focos
total = ds['focos'].salem.roi(shape=shapefile_brasil)
total = int(np.sum(total))

# plota mapa
map2 = ax[1].contourf(ds['lon'],
                      ds['lat'],
                      ds['focos'].salem.roi(shape=shapefile_brasil),
                      alpha=1.0,
                      cmap='lajolla',
                      vmin=0.1, vmax=160,
                      levels=np.array([0.1, 5, 10, 15, 20, 30, 40, 50, 60, 70, 100, 130, 160]),
                      extend='max',
                      colorbar='best',
                      colorbar_kw={'label': 'Satelite: AQUA/ Resolução Mapa: 20km \nFonte Dados: INPE',
                                   'length': 18,
                                   'frameon': False,
                                   'ticklabelsize': 10,
                                   'labelsize': 7,
                                   'width': 2,
                                   'ticks': [20, 50, 100, 150],
                                   'ticklabels': ['20', '50', '100', '150']})

# plota subtítulo
ax[1].format(title=anos[1],
             titleloc='c',
             titleweight='bold',
             titlecolor='gray')

# plota contorno dos Estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/refs/heads/main/estados_do_brasil/BR_UF_2019.shp').geometries())
ax[1].add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='gray', facecolor='none', linewidth=1.0, alpha=1.0)

# plota contorno do Brasil
shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp').geometries())
ax[1].add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.0, alpha=1.0)

# legenda
ax[1].legend(loc='ll', ncols=1, frameon=False, prop={'size': 12.}, markerscale=1)

# plota quantidade de focos de calor na legenda
ax[1].text(-46, -30., f'Total={total}', color='red', fontsize=15)
#--------------------------------------------------------------#

# salva figura
fig.savefig(f'{dir_output}/Parte_4_Fig_acumulado_focos_calor_Brasil_{str(anos[0])}_e_{str(anos[1])}.jpg', transparent=True, dpi=300, bbox_inches="tight")

---

>`TAREFA:`

- Escolha dois outros anos da série histórica (2003 à 2025) e realize uma comparação entre as diferenças na quantidade de focos por ano e distribuição espacial.

---



# **PARTE 5):** Mapa de Acumulado e Anomalia de Focos de Calor

## Acumula na grade

In [ ]:
%%time
#========================================================================================================================#
#                                          CRIA DIRETÓRIO DE SAÍDA
#========================================================================================================================#
dir_output = '/content/output/Parte_5'
os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                        DEFINE A MATRIZ DE DADOS
#========================================================================================================================#
# limites do Brasil
lonmin, lonmax, latmin, latmax = -75.0, -33.0, -35.0, 7.0

# espaçamento da grade
delta = 20/100.   # grade com 20 km de resolução espacial

# montando a grade
lons = np.arange(lonmin, lonmax, delta)
lats = np.arange(latmax, latmin, -delta)

# quantidade de pontos para longitude e latitude
nlon = len(lons)
nlat = len(lats)

#========================================================================================================================#
#                                                  LOOP NOS ANOS
#========================================================================================================================#
# Loop dos anos
for ano in np.arange(2003,2026):

    print('Processando ===>>>', ano)

    # seleciona apenas as linhas do DataFrame daquele ANO
    df_selec = df.loc[f'{str(ano)}']

    # gera matriz de focos de calor
    focos_lon, focos_lat = df_selec['lon'].values, df_selec['lat'].values

    # interpolando para ponto de grade
    focos = np.zeros((nlat, nlon))

    # loop em cada longitude e latitude da lista
    for lonfoco, latfoco in zip(focos_lon, focos_lat):

        # função que extrai a qual pixel aquele relâmpago pertence
        lin, col = index(lons, lats, lonfoco, latfoco)

        # soma os relâmpagos por pixel
        focos[lin,col]+=1
    print('.... Gerou a matriz de focos de calor')

    # gera arquivo netcdf
    data_vars = {'focos':(('lat', 'lon'), focos, {'units': 'ocorrências/400km²', 'long_name':'Focos de Calor'})}
    coords = {'lat': lats, 'lon': lons, 'time': pd.to_datetime(f'{str(ano)}-12')}
    ds = xr.Dataset(data_vars=data_vars, coords=coords)
    ds.to_netcdf(f'{dir_output}/Parte_5_focos_anual_Brasil_AQUA_{ano}.nc')
    print('.... Gerou e salvou arquivo netcdf','\n')

## Agrupa os dados

In [ ]:
# define ano
ano = '2025'

# lista dos arquivos
files = sorted(glob.glob(f'{dir_output}/Parte_5_focos_anual_Brasil_*.nc')) # focos_anual_brasil_AQUA_2024.nc

# leitura dos arquivos por ano
focos_por_ano = xr.open_mfdataset(files, concat_dim='time', combine='nested')

# coloca "NaN" onde os "focos=0"
condicao = focos_por_ano['focos'][:,:,:] == 0
focos_por_ano['focos'][:,:,:] = np.where(condicao, np.nan, focos_por_ano['focos'][:,:,:])

# calculando a CLIMATOLOGIA: 2003-2023
climatologia = focos_por_ano.sel(time=slice('2003-01-01','2023-12-31')).mean("time")

# calculando a ANOMALIA para o ano definido acima
anomalia = focos_por_ano['focos'].sel(time=slice(ano,ano))[0,:,:] - climatologia

# recorta o dado para região
anomalia = anomalia.salem.roi(shape=shapefile_brasil)
climatologia = climatologia.salem.roi(shape=shapefile_brasil)

In [ ]:
# mostra os dados de focos por ano
focos_por_ano

In [ ]:
# mostra os dados da climatologia
climatologia

In [ ]:
# mostra os dados da anomalia
anomalia

## Plota figura

In [ ]:
%%time
#========================================================================================================================#
#                                           PLOTA FIGURA
#========================================================================================================================#
# cria a moldura da figura
fig, ax = uplt.subplots(axwidth=6, ncols=2, nrows=1, tight=True, proj='pcarree')

# limites do Brasil
lonmin, lonmax, latmin, latmax = -75.0, -33.0, -35.0, 7.0

# define formato da figura
ax.format(coast=False, borders=False, innerborders=False,
          labels=False, latlines=5, lonlines=10,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          small='20px', large='25px',
          linewidth=0, grid=False)

#--------------------------------------------------------------#
#                            FIG-1
#--------------------------------------------------------------#
# plota mapa
map1 = ax[0].contourf(focos_por_ano['lon'],
                      focos_por_ano['lat'],
                      focos_por_ano['focos'].loc[f'{ano}-12-01'].salem.roi(shape=shapefile_brasil),
                      cmap='lajolla',
                      vmin=0, vmax=160,
                      levels=np.array([0, 5, 10, 15, 20, 30, 40, 50, 60, 70, 100, 130, 160]),
                      extend='max',
                      colorbar='best',
                      colorbar_kw={'label': 'Satelite: AQUA/ Resolução Mapa: 20km \nFonte Dados: INPE',
                                   'length': 18,
                                   'frameon': False,
                                   'ticklabelsize': 10,
                                   'labelsize': 7,
                                   'width': 2,
                                   'ticks': [20, 50, 100, 150],
                                   'ticklabels': ['20', '50', '100', '150']})

# título da figura
ax[0].format(title=f'Acumulado de Focos',
             titleloc='l',
             titleweight='bold',
             titlecolor='black')

# plota subtítulo
total = focos_por_ano['focos'].loc[f'{ano}-12-01'].salem.roi(shape=shapefile_brasil) ; total = int(np.sum(total))
ax[0].text(lonmin-0.5, latmax-1.0, f'Período={ano} / Total de focos={total}', color='grey', fontsize=10)

# plota contorno dos Estados
estados_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/estados_do_brasil/BR_UF_2019.shp')
estados_brasil.plot(edgecolor='gray', facecolor='none', linewidth=1.0, alpha=1, ax=ax[0])

# plota contorno do Brasil
contorno_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp')
contorno_brasil.plot(edgecolor='black', facecolor='none', linewidth=1.0, alpha=1, ax=ax[0])

#--------------------------------------------------------------#
#                            FIG-2
#--------------------------------------------------------------#
ax[1].contourf(anomalia['lon'],
               anomalia['lat'],
               anomalia['focos'][:,:],
               cmap='coolwarm',
               vmin=-20, vmax=20,
               levels=uplt.arange(-20, 20, 5),
               extend='both',
               colorbar='best',
               colorbar_kw={'label': 'Satelite: AQUA/ Resolução Mapa: 20km \nFonte Dados: INPE',
                            'length': 18,
                            'frameon': False,
                            'ticklabelsize': 10,
                            'labelsize': 7,
                            'width': 2,
                            'ticks': 10})

# título da figura
ax[1].format(title=f'Anomalia de Focos',
             titleloc='l',
             titleweight='bold',
             titlecolor='black')

# plota subtítulo
total = anomalia['focos'][:,:] ; total = int(np.max(total))
ax[1].text(lonmin-0.5, latmax-1.0, f'Período={ano} / Máxima Anomalia={total}', color='grey', fontsize=10)

# plota contorno dos Estados
estados_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/estados_do_brasil/BR_UF_2019.shp')
estados_brasil.plot(edgecolor='gray', facecolor='none', linewidth=1.0, alpha=1, ax=ax[1])

# plota contorno do Brasil
contorno_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp')
contorno_brasil.plot(edgecolor='black', facecolor='none', linewidth=1.0, alpha=1, ax=ax[1])

# salva figura
fig.save(f'{dir_output}/Parte_5_acumulado_e_anomalia_focos_{ano}.png', dpi=300)

# **PARTE 6):** Séries Temporais de Focos de Calor

Analisaremos a série histórica de focos de calor de 2003 à 2025. Iremos gerar o gráfico de focos por dia, focos por ano e focos por ano/mês com o CalendarPlot.

## Figura: `Total Diária`

Agrupa os dados por dia.
- No exemplo dos focos de calor diário, gostaríamos de calcular a quantidade de focos de calor que ocorreu para cada dia, desde `1 de janeiro de 2010` até `04 de novembro de 2025` para o estado de `Minas Gerais (MG)`.
- Para isto usaremos a função **groupby**, e como argumento utilizaremos a função [**Grouper**](https://pandas.pydata.org/docs/reference/api/pandas.Grouper.html) do pandas. Usaremos o argumento **Freq='1D'** para dizer que queremos agrupar os dados por dia.

In [ ]:
# mostrando os dados que utilizaremos
df

In [ ]:
#========================================================================================================================#
#                                             CRIA DIRETÓRIO DE SAÍDA
#========================================================================================================================#
dir_output = '/content/output/Parte_6'
os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                 SELECIONA FOCOS PARA O ESTADO E AGRUPA POR DIA
#========================================================================================================================#
# seleciona os focos ocorridos em MG
df_estado = df[ df['estado'] == 'MINAS GERAIS' ]

# calculando a quantidade de focos de calor por dia
diaria = df_estado.groupby(pd.Grouper(freq='1D')).count()['lat']

#========================================================================================================================#
#                                                 PLOTA FIGURA
#========================================================================================================================#
# cria a moldura da figura
fig, ax = uplt.subplots(figsize=(10,5), tight=True)

# extrai a data inicial e final do dataframe
anoi, mesi, diai = str(diaria.index[0].year), str(diaria.index[0].month).zfill(2), str(diaria.index[0].day).zfill(2)
anof, mesf, diaf = str(diaria.index[-1].year), str(diaria.index[-1].month).zfill(2), str(diaria.index[-1].day).zfill(2)

# plota gráfico de barras
ax.bar(diaria.index,
       diaria.values,
       color='red')

# formato dos eixos
ax.format(title=f'Focos de Calor\n',
          titleloc='l',
          titleweight='bold',
          titlecolor='bright red',
          xlabel='Ano',
          ylabel='Focos de Calor / Dia',
          small='20px',
          large='20px',
          abc=False,
          xrotation=0,
          ytickminor=False,
          xtickminor=False)

# calcula os valores dos percentis
p95 = np.percentile(diaria.values, 95)
p99 = np.percentile(diaria.values, 99)

# plota as linhas dos percentis
ax.axhline(p95, color='black', lw=1.0, linestyle='--')
ax.axhline(p99, color='black', lw=1.0, linestyle='--')

# coloca o texto na ultima data + um pequeno offset, usando coordenadas de datas no eixo X**
ultima_data = diaria.index[-1]
offset_dias = pd.Timedelta(days=2)

# adiciona textos próximos aos valores
ax.text(ultima_data + offset_dias, p95, 'P95%', color='black', fontsize=9,
        verticalalignment='bottom', horizontalalignment='left')
ax.text(ultima_data + offset_dias, p99, 'P99%', color='black', fontsize=9,
        verticalalignment='bottom', horizontalalignment='left')

# plota subtítulo
ax.text(0.000, 1.04,
        f'Satélite: AQUA (1km) | Período: {anoi}/{mesi}/{diai} à {anof}/{mesf}/{diaf} | Estatística: Diária | Estado: Minas Gerais | Fonte Dados: INPE',
        transform=ax.transAxes,
        color='gray',
        fontsize=9,
        verticalalignment='top')

# salva figura
fig.save(f'{dir_output}/Parte_6_Fig_1_focos_por_dia.jpg', dpi=300)

In [ ]:
# mostrando a diária total
diaria

## Figura: `Heatmap`

In [ ]:
#========================================================================================================================#
#                                             CRIA DIRETÓRIO DE SAÍDA
#========================================================================================================================#
dir_output = '/content/output/Parte_6'
os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                                  AGRUPA OS DADOS
#========================================================================================================================#
# seleciona os focos de calor em MG
df_estado = df[df['estado']=='MINAS GERAIS']

# agrupa os dados por "MÊS"
#df_mes = df.groupby('estado').get_group('MINAS GERAIS').groupby(pd.Grouper(freq='1M')).count()['estado']
df_mes = df_estado.groupby(pd.Grouper(freq='1M')).count()['estado']

# preenche com zeros os meses de 2025 que ainda não chegaram
ano_mes_dia = []

ano_mesi, ano_mesf = f'{str(anoi)}0101', f'{str(2026)}1231'

for data in pd.date_range(ano_mesi, ano_mesf, freq='1M'):
    ano_mes_dia.append(data.strftime('%Y-%m-%d'))

datas = np.array(ano_mes_dia)

df_mes = df_mes.reindex(datas, fill_value=np.nan)

# matriz com formato de anos x meses
focos_table = np.reshape(df_mes.values, ((2026-2003)+1, 12), order='C')

#========================================================================================================================#
#                                               PLOTA FIGURA
#========================================================================================================================#
#========================================================#
#               DEFINIÇÕES INICIAIS
#========================================================#
# moldura da figura
fig, ax = plt.subplots(figsize=(10,10))

# criando heatmap com seaborn
sns.heatmap(focos_table,
            vmin=0.1, vmax=3000,
            cmap='lajolla',
            xticklabels=['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'],
            yticklabels=uplt.arange(2003, 2026, 1),
            linewidth=0.5,
            linecolor='white',
            cbar_kws={'label': ' ',
                      'shrink': 1.0,
                      'pad': 0.01,
                      'orientation': 'vertical'},
            annot=True, fmt=".0f",
            annot_kws={'color': 'gray',
                       'fontsize': 13,
                       'fontweight': 'medium'})

# configurações da barra de cores
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=15, axis="both")
cbar.set_label('Satélite: AQUA / Resolução Dados: 1km\nFonte Dados: INPE', fontsize=14)
cbar.ax.minorticks_off()

# título
anoix, mesix, diaix = str(df.head(1).index.year[0]), str(df.head(1).index.month[0]), str(df.head(1).index.day[0])
anofx, mesfx, diafx = str(df.tail(1).index.year[0]), str(df.tail(1).index.month[0]), str(df.tail(1).index.day[0])
ax.set_title('Focos de Calor em Minas Gerais', fontsize=16, color='red', fontweight='bold', loc='left')
ax.set_title(f'Período: {anoix}/{mesix.zfill(2)}/{diaix.zfill(2)} à {anofx}/{mesfx.zfill(2)}/{diafx.zfill(2)}', fontsize=12, color='gray', loc='right')

# retirar os minorticks
ax.minorticks_off()

# retirar a grade
plt.grid(False)

# orientações labels do eixo Y
plt.yticks(rotation=0, fontsize=13)
plt.xticks(rotation=0, fontsize=13)

# salva figura
plt.tight_layout()
plt.savefig(f'{dir_output}/Parte_6_Fig_2_heatmap.jpg', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# mostra os dados
#df_mes

## Figura: `Por Ano`

In [ ]:
# dados que utilizaremos
df_estado

In [ ]:
#========================================================================================================================#
#                                             CRIA DIRETÓRIO DE SAÍDA
#========================================================================================================================#
dir_output = '/content/output/Parte_6'
os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                            AGRUPA OS DADOS POR ANO
#========================================================================================================================#
# calculando a anual total
anual = df_estado.groupby(pd.Grouper(freq='1Y')).count()['lat']

#========================================================================================================================#
#                                                 PLOTA FIGURA
#========================================================================================================================#
# cria moldura da figura
fig, ax = uplt.subplots(figsize=(10, 5), tight=True)

# plota gráfico de barras
ax.bar(anual.index.year,
       anual.values,
       color='blue',
       alpha=0.5)

# insere os valores de anomalia no topo das barras
anomalias = ((anual.values - np.mean(anual.values)) / np.mean(anual.values)) * 100.
for ano_x, focos_x, anomalia_x in zip(anual.index.year, anual.values, anomalias):
    val_x = f'{str(int(anomalia_x))}%'
    if anomalia_x > 0.:
        cor = 'bright red'
    else:
        cor = 'bright blue'
    ax.text(ano_x-0.39, focos_x, val_x, color=cor, fontsize=11, weight='bold')


# calcula os valores dos percentis
p95 = np.percentile(anual.values[0:-1], 95)
p99 = np.percentile(anual.values[0:-1], 99)

# plota as linhas dos percentis
ax.axhline(p95, color='gray', lw=0.7, linestyle='--')
ax.axhline(p99, color='gray', lw=0.7, linestyle='--')

# adiciona textos próximos aos valores
ax.text(2026, p95, 'P95%', color='gray', fontsize=9,
        verticalalignment='bottom', horizontalalignment='left')
ax.text(2026, p99, 'P99%', color='gray', fontsize=9,
        verticalalignment='bottom', horizontalalignment='left')

# formato dos eixos
ax.format(title=f'Focos de Calor\n',
          titleloc='l',
          titleweight='bold',
          titlecolor='bright red',
          xlabel='Ano',
          ylabel='Focos de Calor / Ano',
          small='18px',
          large='20px',
          abc=False,
          xrotation=50,
          ytickminor=False,
          xtickminor=False,
          xticks=uplt.arange(2003,2026,1))

# plota subtítulo
ax.text(0.000, 1.04,
        f'Satélite: AQUA (1km) | Período: {anoi}/{mesi}/{diai} à {anof}/{mesf}/{diaf} | Estatística: Anual | Estado: Minas Gerais | Fonte Dados: INPE',
        transform=ax.transAxes,
        color='gray',
        fontsize=9,
        verticalalignment='top')

# salva figura
fig.save(f'{dir_output}/Parte_6_Fig_3_focos_por_ano.jpg', dpi=300)

In [ ]:
# mostrando a anual total
anual

## Figura: `CALENDARPLOTS`

In [ ]:
# dados que utilizaremos
diaria

In [ ]:
# selecionando os dados de 2017
diaria.loc['2017']

In [ ]:
#========================================================================================================================#
#                                             CRIA DIRETÓRIO DE SAÍDA
#========================================================================================================================#
dir_output = '/content/output/Parte_6'
os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                             DEFINIÇÕES INICIAIS
#========================================================================================================================#
# moldura da figura
fig, ax = plt.subplots(figsize=(9,8), nrows=4)

# labels dos "dias da semana" e "meses do ano"
dias = ['seg', 'ter', 'qua', 'qui', 'sex','sáb', 'dom']
meses = ['JAN', 'FEV', 'MAR', 'ABR', 'MAI', 'JUN', 'JUL', 'AGO', 'SET', 'OUT', 'NOV', 'DEZ']

#========================================================#
#                 FIGURA A) - 2023
#========================================================#
map1 = calplot.yearplot(diaria.loc['2023'],
                        cmap='Oranges1',
                        daylabels=dias,
                        monthlabels=meses,
                        vmin=0,
                        vmax=100,
                        ax=ax[0])

#========================================================#
#                  FIGURA B) - 2024
#========================================================#
calplot.yearplot(diaria.loc['2024'],
                 cmap='Oranges1',
                 daylabels=dias,
                 monthlabels=meses,
                 vmin=0,
                 vmax=100,
                 ax=ax[1])

#========================================================#
#                  FIGURA C) - 2025
#========================================================#
calplot.yearplot(diaria.loc['2025'],
                 cmap='Oranges1',
                 daylabels=dias,
                 monthlabels=meses,
                 vmin=0,
                 vmax=100,
                 ax=ax[2])

#========================================================#
#                  FIGURA D) - 2026
#========================================================#
calplot.yearplot(diaria.loc['2026'],
                 cmap='Oranges1',
                 daylabels=dias,
                 monthlabels=meses,
                 vmin=0,
                 vmax=100,
                 ax=ax[3])

#========================================================#
#                  DEMAIS DEFINIÇÕES
#========================================================#
# barra de cores
cb = fig.colorbar(map1.get_children()[1], orientation='horizontal', ticks=uplt.arange(0, 100, 20), shrink=0.75, ax=ax[3])
cb.ax.tick_params(labelsize=12)
cb.ax.tick_params(which='minor', width=0)
cb.set_label('Quantidade de Focos de Calor', fontsize=14)

# textos
ax[0].text(1., 9., 'Focos de Calor em Minas Gerais', fontsize=16, color='red', fontweight='bold')
ax[0].text(36., 9., 'Satélite: AQUA / Fonte Dados: INPE', fontsize=12, color='gray')
ax[0].text(1., 7.2, f"2023 - Total: {np.sum(diaria.loc['2023'].values)}", fontsize=12, color='black')
ax[1].text(1., 7.2, f"2024 - Total: {np.sum(diaria.loc['2024'].values)}", fontsize=12, color='black')
ax[2].text(1., 7.2, f"2025 - Total: {np.sum(diaria.loc['2025'].values)}", fontsize=12, color='black')
ax[3].text(1., 7.2, f"2026 - Total: {np.sum(diaria.loc['2026'].values)}", fontsize=12, color='black')

# salva figura
plt.tight_layout()
plt.savefig(f'{dir_output}/Parte_6_Fig_4_calendarplot.jpg', bbox_inches='tight', dpi=300)
plt.show()

# **PARTE 7):** Risco de Fogo Observado

In [ ]:
#========================================================================================================================#
#                                             CRIA DIRETÓRIO DE SAÍDA
#========================================================================================================================#
dir_output = '/content/output/Parte_7/'
os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                              DEFINE O ANO, MÊS E DIA
#========================================================================================================================#
ano, mes, dia = '2026', '02', '04'

#========================================================================================================================#
#                                               DOWNLOAD DO ARQUIVO
#========================================================================================================================#
# nome do arquivo
file = f'INPE_FireRiskModel_2.2_FireRisk_{ano}{mes}{dia}.nc'

# caminho completo para salvar o arquivo
filepath = os.path.join(dir_output, file)

# URL direta para o arquivo
url = f'https://dataserver-coids.inpe.br/queimadas/queimadas/riscofogo_meteorologia/observado/risco_fogo/{file}'

# baixa o arquivo
print(f"Baixando: {url}")
response = requests.get(url, stream=True, verify=False)

if response.status_code == 200:
    with open(filepath, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Arquivo baixado com sucesso: {filepath}")
    print(f"Tamanho do arquivo: {os.path.getsize(filepath)} bytes")
else:
    print(f"Erro ao baixar. Status: {response.status_code}")

#========================================================================================================================#
#                                                   PLOTA FIGURA
#========================================================================================================================#
# limites do Brasil
lonmin, lonmax, latmin, latmax = -75.0, -33.0, -35.0, 7.0

# leitura do shapefile do Brasil
shapefile_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp')

# cria a moldura da figura
fig, ax = uplt.subplots(axwidth=6, ncols=1, nrows=1, tight=True, proj='pcarree')

# define formato da figura
ax.format(coast=False, borders=False, innerborders=False,
          labels=True, latlines=10, lonlines=10,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          small='17px', large='20px',
          linewidth=1.0, grid=True,
          suptitle='Risco de Fogo Observado')

# leitura do arquivo netcdf
ds = xr.open_dataset(f'{dir_output}/{file}').sel(lat=slice(latmin,latmax), lon=slice(lonmin,lonmax))

# coloca "NaN" onde o "risco=0"
condicao = ds['rf'][:,:,:] == 0
ds['rf'][:,:,:] = np.where(condicao, np.nan, ds['rf'][:,:,:])

# plota mapa
map1 = ax.contourf(ds['lon'],
                   ds['lat'],
                   ds['rf'][0,:,:].salem.roi(shape=shapefile_brasil),
                   alpha=1.0,
                   cmap='jet',
                   levels=uplt.arange(0.0, 1.0, 0.1),
                   extend='max',
                   colorbar='best',
                   colorbar_kw={'label': 'Fonte Dados: INPE / Resolução Mapa: 1km',
                                'length': 16,
                                'frameon': False,
                                'ticklabelsize': 10,
                                'labelsize': 7,
                                'width': 1.5,
                                'ticks': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
                                'ticklabels': ['0.0', '0.2', '0.4', '0.6', '0.8', '1.0']})

# plota subtítulo
ax.format(title=f'Período: {ano}-{mes}-{dia}',
          titleloc='c',
          titleweight='light',
          titlecolor='gray',
          titlesize=13)

# plota contorno dos Estados
shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/refs/heads/main/estados_do_brasil/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='gray', facecolor='none', linewidth=0.3, alpha=1.0)

# plota contorno do Brasil
shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.0, alpha=1.0)

# salva figura
fig.savefig(f'{dir_output}/Parte_7_risco_fogo_observado_{ano}{mes}{dia}.jpg', transparent=True, dpi=300, bbox_inches="tight")

In [ ]:
# mostra os dados
ds

# **PARTE 8):** Risco de Fogo Previsto

In [ ]:
%%time

#========================================================================================================================#
#                                             CRIA DIRETÓRIO DE SAÍDA
#========================================================================================================================#
dir_output = '/content/output/Parte_8/'
os.makedirs(dir_output, exist_ok=True)

#========================================================================================================================#
#                                               DOWNLOAD DO ARQUIVO
#========================================================================================================================#
# nomes dos arquivos
files = ['RF.PREV.T0.tif', 'RF.PREV.T1.tif', 'RF.PREV.T2.tif', 'RF.PREV.T3.tif',]

# loop nos arquivos
for file in files:

    # URL dos dados
    url = f"https://dataserver-coids.inpe.br/queimadas/queimadas/riscofogo_meteorologia/previsto/risco_fogo/{file}"
    caminho = f"{dir_output}{file}"

    # baixando os arquivos
    with open(caminho, "wb") as f:
        f.write(requests.get(url, verify=False).content)
    print(f"✓ {file}")

#========================================================================================================================#
#                                                   PLOTA FIGURA
#========================================================================================================================#
# limites do Brasil
lonmin, lonmax, latmin, latmax = -75.0, -33.0, -35.0, 7.0

# leitura do shapefile do Brasil
shapefile_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp')

# cria a moldura da figura
fig, ax = uplt.subplots(axwidth=6, nrows=1, ncols=4, tight=True, proj='pcarree')

# define formato da figura
ax.format(coast=False, borders=False, innerborders=False,
          labels=True, latlines=10, lonlines=10,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          small='25px', large='30px',
          linewidth=1.0, grid=True,
          suptitle='Risco de Fogo Previsto')

#--------------------------------------------------------------#
#                   LOOP DOS 4 TEMPOS
#--------------------------------------------------------------#
# loop dos tempos
for i, file in enumerate(files):

    # leitura do arquivo netcdf
    ds = xr.open_dataset(f'{dir_output}/{file}', engine='rasterio').sel(y=slice(latmax,latmin), x=slice(lonmin,lonmax))

    # coloca "NaN" onde o "risco=0"
    condicao = ds['rbf'][:,:,:] == 0 ; ds['rbf'][:,:,:] = np.where(condicao, np.nan, ds['rbf'][:,:,:])

    # extraindo a data do arquivo
    data = ds.time.values[0]
    ano, mes, dia, hor = str(data.year), str(data.month).zfill(2), str(data.day).zfill(2), str(data.hour).zfill(2)

    # salva a primeira data
    if (i==0): datai = ano+mes+dia

    # plota mapa
    map1 = ax[i].contourf(ds['x'],
                          ds['y'],
                          ds['rbf'][0,:,:].salem.roi(shape=shapefile_brasil),
                          cmap='jet',
                          vmin=0.0, vmax=1.0,
                          levels=uplt.arange(0.0, 1.0, 0.1),
                          extend='max')

    # plota contorno dos Estados
    shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/refs/heads/main/estados_do_brasil/BR_UF_2019.shp').geometries())
    ax[i].add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='gray', facecolor='none', linewidth=0.3, alpha=1.0)

    # plota contorno do Brasil
    shapefile = list(shpreader.Reader('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp').geometries())
    ax[i].add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=1.0, alpha=1.0)

    # plota subtítulo
    if (i==0): ax[i].format(title=f'{ano}-{mes}-{dia}', labels=[True, False, True, False], linewidth=1, titleloc='c', titlecolor='grey', titlesize=20)
    if (i==1): ax[i].format(title=f'{ano}-{mes}-{dia}', labels=[False, False, True, False], linewidth=1, titleloc='c', titlecolor='grey', titlesize=20)
    if (i==2): ax[i].format(title=f'{ano}-{mes}-{dia}', labels=[False, False, True, False], linewidth=1, titleloc='c', titlecolor='grey', titlesize=20)
    if (i==3): ax[i].format(title=f'{ano}-{mes}-{dia}', labels=[False, False, True, False], linewidth=1, titleloc='c', titlecolor='grey', titlesize=20)

# plota barra de cores
fig.colorbar(map1, loc='b', label='Fonte Dados: INPE / Resolução Mapa: 1km', ticks=0.1, ticklabelsize=20, labelsize=20, width=0.4, length=0.4)

# salva figura
fig.savefig(f'{dir_output}/Parte_8_risco_fogo_previsto_a_partir_de_{datai}.jpg', dpi=300, bbox_inches="tight")

# **Parabéns**, você chegou ao final da primeira aula do curso sobre queimadas !!! 👏 👏 👏

Agradecemos a sua participação e até o próximo curso. E lembre-se, `Python é vida` !!!

![Texto alternativo](https://github.com/evmpython/Minicurso_queimadas_UNIFEI_INPE_2026/blob/main/04_logos/FINAL.jpg?raw=true)